In [1]:
import  pandas as pd
import numpy as np

In [5]:
# 1. 파일 로드
path = r"C:\Users\MetaM\PycharmProjects\investment\data\DataGuide_Ratio.xlsx"
raw_df = pd.read_excel(path, sheet_name='PSR', header=None)

# 2. Symbol, Name, Indicator 추출
symbol_row = 7
name_row = 8
item_name_row = 11
start_data_row = 13

symbols = raw_df.iloc[symbol_row, 2:].values
names = raw_df.iloc[name_row, 2:].values
indicators = raw_df.iloc[item_name_row, 2:].values

# 3. 날짜 데이터 추출 + to_datetime 처리 (오류 무시)
raw_dates = raw_df.iloc[start_data_row:, 0]
parsed_dates = pd.to_datetime(raw_dates, errors='coerce')  # 에러 무시하고 NaT로

# 4. long format으로 정리
records = []
for col_idx, symbol in enumerate(symbols):
    if pd.isna(symbol):
        continue
    name = names[col_idx]
    indicator = indicators[col_idx]
    values = raw_df.iloc[start_data_row:, col_idx + 2].values
    for date, value in zip(parsed_dates, values):
        if pd.isna(date) or pd.isna(value):
            continue
        records.append({
            'date': date,
            'symbol': symbol,
            'name': name,
            'indicator': indicator,
            'value': value
        })

# 5. 결과 DataFrame
clean_df = pd.DataFrame(records)
print(clean_df.head())


        date             symbol     name   indicator    value
0 2004-01-31  Current(20250605)  A000660  M001790002  0.76583
1 2004-02-29  Current(20250605)  A000660  M001790002  0.76754
2 2004-03-31  Current(20250605)  A000660  M001790002  1.04821
3 2004-04-30  Current(20250605)  A000660  M001790002  0.78614
4 2004-05-31  Current(20250605)  A000660  M001790002  0.74320


C:\Users\MetaM\AppData\Local\Temp\ipykernel_19720\896053432.py:17: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  parsed_dates = pd.to_datetime(raw_dates, errors='coerce')  # 에러 무시하고 NaT로
